In [3]:
#first take it easy

#lets define dtw function
from fastdtw import fastdtw

def dtw(x, y):
    distance, path = fastdtw(x, y)
    return distance


#see, simple

In [4]:
#now lets define lb_Keogh function

import numpy as np

def lb_Keogh(s1,s2,r):
    #s2 is the query, s1 is the train
    
    LB_sum=0
    for ind,i in enumerate(s1):
        
        lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        
        if i>upper_bound:
            LB_sum=LB_sum+(i-upper_bound)**2
        elif i<lower_bound:
            LB_sum=LB_sum+(i-lower_bound)**2
            
    return np.sqrt(LB_sum)

In [5]:
#now lets see how to index the time series

#first make a paa function
def paa(x, w):
    #w is the number of segments
    #first calculate the length of the time series
    n = len(x)
    #now calculate the length of each segment
    step = n/w
    #now calculate the paa
    paa = []
    for i in range(w):
        paa.append(np.mean(x[int(i*step):int((i+1)*step)]))
    return paa

x = [1,2,3,4,5,6,7,8,9,10,11,12]
w = 4
print(paa(x,w))

#now lets make a function to index the time series
def paa_upper_lower(ts, w, r):
    #ts is the time series, w is the window size, r is the radius
    #first calculate the upper and lower bound of the ts
    lower_bound=[]
    upper_bound=[]
    
    for ind,i in enumerate(ts):
        lower_bound.append(min(ts[(ind-r if ind-r>=0 else 0):(ind+r)]))
        upper_bound.append(max(ts[(ind-r if ind-r>=0 else 0):(ind+r)]))
    
    
    #now calculate the paa of upper and lower bound
    lower_bound_paa=paa(lower_bound,w)
    upper_bound_paa=paa(upper_bound,w)
    
    return lower_bound_paa, upper_bound_paa

def lb_paa(Q,C,w):
    L,U = paa_upper_lower(Q,len(Q),1)
    
    sum = 0
    for i in range(0,len(C),w):
        if Q[i] > U[i]:
            sum = sum + (C[i]-U[i])**2
        elif Q[i] < L[i]:
            sum = sum + (C[i]-L[i])**2
            
    return np.sqrt(sum)


[2.0, 5.0, 8.0, 11.0]


In [6]:
# #see, its also simple

# #now lets define kNN function

# from collections import Counter

# def knn(train,test,r,k):
#     #r is the size of the window
#     #k is the number of nearest neighbors
    
#     dist=[]
    
#     for ind,i in enumerate(train):
#         if len(i)==len(test):
#             dist.append(dtw_distance(i,test))
#         else:
#             dist.append(lb_Keogh(i,test,r))
            
#     ids=np.argsort(dist)[:k]
    
#     #now we will take the most common class among these k nearest neighbors
#     #and return it as our prediction
    
#     count=Counter()
#     for i in ids:
#         count[train[i][0]]+=1
        
#     return count.most_common()[0][0]


In [7]:
from rtree import index
import os

# Create a 4-dimensional index
properties = index.Property(dimension=4)
four_dimensional_index = index.Index(properties=properties)

# Insert data into the index
j = 0
for filename in os.listdir("test_dataset/UCR"):
    f = open("test_dataset/UCR/" + filename + "/" + filename + "_TEST", "r")
    content = f.read()
    content = content.split("\n")
    content = content[0].split(",")
    content = content[1:]
    content = [float(i) for i in content]

    # Make a paa of the content with window size 4
    content_paa = paa(content, 4)

    # Now insert the content_paa into the index
    four_dimensional_index.insert(j, content_paa,obj=filename)
    j = j + 1








# f = open("query_samples/Adiac_TRAIN", "r")
# content = f.read()
# content = content.split("\n")
# content = content[0].split(",")
# content = content[1:]
# content = [float(i) for i in content]

T = paa(content, 4)

# L, U = paa_upper_lower(content, 4, 1)

# query_window = tuple(T)

# # Perform the query
# result = list(four_dimensional_index.nearest(query_window,5,objects=True))

# # Print the result
# for i in result:
#     print(i.object)






# print("Query window:", query_window)

# # Perform the query for nearest neighbors
# result = list(four_dimensional_index.nearest(query_window, 5))

# # Print the result
# print("Result:", result)


In [8]:
import random
import os 

class Rectangle:
    def __init__(self, index, xmin, ymin, xmax, ymax, data_object=None):
        self.index = index
        self.xmin, self.ymin, self.xmax, self.ymax = xmin, ymin, xmax, ymax
        self.data_object = data_object

class Node:
    def __init__(self, is_leaf=True):
        self.is_leaf = is_leaf
        self.children = []
        self.rectangles = []
        


class RTree:
    
    def __init__(self, max_entries=4):
        self.root = Node()
        self.max_entries = max_entries

    def get_root(self):
        return self.root
    
    def is_leaf(self, node):
        return node.is_leaf

    def insert(self, index, xmin, ymin, xmax, ymax, data_object=None, node=None):
        rectangle = Rectangle(index, xmin, ymin, xmax, ymax, data_object)
        if node is None:
            node = self.root
        if node.is_leaf:
            if len(node.rectangles) < self.max_entries:
                node.rectangles.append(rectangle)
            else:
                self.split_leaf(node, rectangle)
        else:
            chosen_child = self.choose_child(node, rectangle)
            self.insert(index, xmin, ymin, xmax, ymax, data_object, node.children[chosen_child])

    def split_leaf(self, leaf, rectangle):
        leaf.rectangles.append(rectangle)
        if len(leaf.rectangles) > self.max_entries:
            split_rectangles = self.split_node(leaf.rectangles)
            new_leaf = Node(is_leaf=True)
            new_leaf.rectangles = split_rectangles[1]
            leaf.rectangles = split_rectangles[0]
            self.insert(rectangle.index, rectangle.xmin, rectangle.ymin, rectangle.xmax, rectangle.ymax, rectangle.data_object, leaf)

    def choose_child(self, node, rectangle):
        # Placeholder: Choose a child based on some criteria (e.g., minimum area increase)
        return random.choice(range(len(node.children)))

    def split_node(self, rectangles):
        # Placeholder: Implement a split algorithm (e.g., linear-cost algorithm)
        # This should return two lists of rectangles after the split
        return rectangles[:len(rectangles)//2], rectangles[len(rectangles)//2:]

    def search(self, query, node=None):
        if node is None:
            node = self.root
        result = []
        for rect in node.rectangles:
            if isinstance(rect, Rectangle) and self.overlap(query, rect):
                if node.is_leaf:
                    result.append(rect)
                else:
                    # Recursively search in child nodes
                    result.extend(self.search(query, rect))
        return result

    def overlap(self, query, rectangle):
        if isinstance(rectangle, Rectangle):
            return (
                query.xmin <= rectangle.xmax and
                query.xmax >= rectangle.xmin and
                query.ymin <= rectangle.ymax and
                query.ymax >= rectangle.ymin
            )
        return False

def calculate_mbr(node):
    if not node.rectangles:
        # Return None for an empty node
        return None

    dimensions = 2  # Assuming 2D for this example, adjust based on the actual number of dimensions
    li = [float('inf')] * dimensions
    hi = [-float('inf')] * dimensions

    for rect in node.rectangles:
        for d in range(dimensions):
            li[d] = min(li[d], rect.xmin)
            hi[d] = max(hi[d], rect.xmax)

    mbr = Rectangle(li[0], li[1], hi[0], hi[1])
    return mbr

           
rtree = RTree(max_entries=4)
for filename in os.listdir("test_dataset/UCR"):
    f = open("test_dataset/UCR/" + filename + "/" + filename + "_TEST", "r")
    content = f.read()
    content = content.split("\n")
    content = content[0].split(",")
    content = content[1:]
    content = [float(i) for i in content]

    # Make a paa of the content with window size 4
    content_paa = paa(content, 4)

    # Now insert the content_paa into the index
    rectangle = Rectangle(index = j, 
                                  xmin = content_paa[0], 
                                  ymin = content_paa[1],
                                  xmax = content_paa[2],
                                  ymax = content_paa[3],
                                  data_object=filename)
    
    rtree.insert(rectangle.index, rectangle.xmin, rectangle.ymin, rectangle.xmax, rectangle.ymax, rectangle.data_object)
    j = j + 1 

print("Root node:", rtree.get_root())

Root node: <__main__.Node object at 0x000001FC2B8D6C80>


In [9]:
query_rectangle = Rectangle(-1, -0.5, -0.8, 1.18, 0.06)  # Adjust the index based on your requirement
result = rtree.search(query_rectangle)

# Displaying the result
print("Query Rectangle:", (query_rectangle.xmin, query_rectangle.ymin),
      (query_rectangle.xmax, query_rectangle.ymax))
print("Result:")
for rect in result:
    print(f"Index: {rect.index}, Data Object: {rect.data_object}, Coordinates: "
          f"({rect.xmin}, {rect.ymin}) - ({rect.xmax}, {rect.ymax})")

Query Rectangle: (-0.5, -0.8) (1.18, 0.06)
Result:
Index: 50, Data Object: 50words, Coordinates: (-0.461969552238806, -0.76471) - (1.1801809850746265, 0.05706313235294119)


In [10]:
def lb_paa(Q,C,w):
    L,U = paa_upper_lower(Q,len(Q),1)
    
    sum = 0
    for i in range(1,w):
        temp = 0
        if Q[i] > U[i]:
            temp = (C[i]-U[i])**2
        elif Q[i] < L[i]:
            temp = (C[i]-L[i])**2
        sum = sum + temp*(len(Q)/w)
            
    return np.sqrt(sum)

def mindist(Q,R,w):
    #Q is the query, R is the minimum bounding rectangle
    
    L,U = paa_upper_lower(Q,len(Q),1)
    sum = 0
    for i in range(1,w):
        temp =0
        if(R[0][i]>U[i]):
            temp = (R[0][i]-U[i])**2
        elif(R[0][i]<L[i]):
            temp = (R[0][i]-L[i])**2
        sum = sum + temp*(len(Q)/w)
    
    return np.sqrt(sum)
        

In [11]:
# from queue import PriorityQueue
# def range_search(root, query, k, w):
#     result = []
#     queue = PriorityQueue()
#     queue.put((0, root))
    
#     while not queue.empty():
#         top = queue.get()
#         node = top[1]
        
#         for entry in node.entries:
#             if entry.isLeaf:
#                 C = entry.data
#                 dtw_distance = dtw(Q, C)  # Replace with your DTW function
#                 if dtw_distance <= top[0]:
#                     result.append((C, dtw_distance))
                    
#                     if len(result) == k:
#                         return result
#             else:
#                 child_node = entry.child
#                 lb = lb_paa(Q, child_node.center, w)
#                 queue.put((lb, child_node))
    
#     return result

def retrieve_full_sequence_from_database(filename):
    f = open("test_dataset/UCR/" + filename + "/" + filename + "_TEST", "r")
    content = f.read()
    content = content.split("\n")
    content = content[0].split(",")
    content = content[1:]
    content = [float(i) for i in content]
    return content

from queue import PriorityQueue

# Assuming you have the Rectangle and RTree classes defined

def range_search(rtree, query, K, w):
    result = []
    queue = PriorityQueue()
    temp = []

    # Step 1
    queue.put(rtree.get_root(),0)

    # Step 2
    while not queue.empty():
        # Step 3
        #get the top element from the queue
        top = queue.get()
        #for each time series C in temp such that DTW(Q,C) <= top.distance
        for item in temp:
            if item[1] <= top[0]:
                result.append(item[0])
                temp.remove(item)
                if len(result) == K:
                    return result
        
        if top.is_leaf:
            # Step 4
            print(top.rectangles)
            for data_item in top.rectangles:
                C = data_item.data_object
                dtw_distance = dtw(query, C)
                
                if dtw_distance <= top[0]:
                    # Step 5
                    temp.remove(C)
                    # Step 6
                    result.append(C)

                    # Step 7
                    if len(result) == K:
                        return result

            # Step 8
            continue

        # Step 9
        if isinstance(top[1].rectangles[0].data_object, Rectangle):
            # Assume the PAA point is stored in the Rectangle's data_object
            C = top[1].rectangles[0].data_object
            full_sequence_C = retrieve_full_sequence_from_database(C)
            temp.append((full_sequence_C, dtw(query, full_sequence_C)))

        # Step 11
        else:
            for data_item in top[1].rectangles:
                C = data_item.data_object
                lb_paa_value = lb_paa(query, C, w)
                mbr = rtree.calculate_mbr(data_item)
                R= []
                R[0][0] = mbr.xmin
                R[0][1] = mbr.ymin
                R[1][0] = mbr.xmax
                R[1][1] = mbr.ymax
                mindist_value = mindist(query, R)  # Replace with your actual MINDIST implementation
                temp.append((C, lb_paa_value + mindist_value))  # Combining LB_PAA and MINDIST

        # Step 12
        temp.sort(key=lambda x: x[1])

        # Step 13
        for item in temp:
            queue.put((item[1], item[0]))
    return result

query_rectangle = Rectangle(-1, -0.5, -0.8, 1.18, 0.06)  # Adjust the index based on your requirement
print(range_search(rtree, query_rectangle, 5, 4))

    

[<__main__.Rectangle object at 0x000001FC2B8D75B0>, <__main__.Rectangle object at 0x000001FC2B8D59F0>, <__main__.Rectangle object at 0x000001FC2B8D7730>, <__main__.Rectangle object at 0x000001FC2B8D7E80>]


TypeError: float() argument must be a string or a real number, not 'Rectangle'

TypeError: float() argument must be a string or a real number, not 'Rectangle'

In [ ]:
#time to make our knn function
from queue import PriorityQueue

def knn(Q,k,idx):
    #Q is the query, k is the number of nearest neighbors, idx is the index
    queue = PriorityQueue()
    temp = []
    result = []
    
    #insert root node of index into the queue
    queue.put((0, idx.get_bounds(), idx))
    
    while(not queue.empty()):
        #get the top element of the queue
        top = queue.get()
        for i in temp:
            if(dtw_distance(Q,i[1])<dtw_distance(Q,top[1])):
                temp.remove(i)
                result.append(i)
                if(len(result)==k):
                    return result
        
        #remove the top element from the queue
        queue.pop()
        
        if 
                

knn(T,5,four_dimensional_index)
    